In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [95]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
full_data = pd.concat([train, test], sort=True)
data_cleaner = [train, test]

In [59]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


In [81]:
full_data.sample(10)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
349,42.0,NaN,S,8.6625,"Dimic, Mr. Jovan",0,350,3,male,0,0.0,315088
451,NaN,NaN,S,19.9667,"Hagland, Mr. Ingvald Olai Olsen",0,452,3,male,1,0.0,65303
102,NaN,NaN,Q,7.7500,"Foley, Mr. William",0,994,3,male,0,NaN,365235
682,20.0,NaN,S,9.2250,"Olsvigen, Mr. Thor Anderson",0,683,3,male,0,0.0,6563
16,2.0,NaN,Q,29.1250,"Rice, Master. Eugene",1,17,3,male,4,0.0,382652
823,27.0,E121,S,12.4750,"Moor, Mrs. (Beila)",1,824,3,female,0,1.0,392096
331,45.5,C124,S,28.5000,"Partner, Mr. Austen",0,332,1,male,0,0.0,113043
486,35.0,C93,S,90.0000,"Hoyt, Mrs. Frederick Maxfield (Jane Anne Forby)",0,487,1,female,1,1.0,19943
274,NaN,NaN,C,7.2250,"Saade, Mr. Jean Nassr",0,1166,3,male,0,NaN,2676
789,46.0,B82 B84,C,79.2000,"Guggenheim, Mr. Benjamin",0,790,1,male,0,0.0,PC 17593


In [80]:
full_data.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [70]:
for row in data_cleaner:
    row['Age'].fillna(row['Age'].median(), inplace = True)
    
print(train.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [83]:
full_data.info()
full_data['Age'] = lambda x: x['Age'].fillna(x['Age'].median(), inplace = True)
full_data['Embarked'] = lambda x: x['Embarked'].fillna(x['Embarked'].mode(), inplace = True)
full_data['Fare'] = lambda x: x['Fare'].fillna(x['Fare'].median(), inplace = True)
full_data.drop(['Cabin', 'PassengerId', 'Ticket'], axis=1, inplace=True)
full_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1309 non-null object
Cabin          295 non-null object
Embarked       1309 non-null object
Fare           1309 non-null object
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 132.9+ KB


Age           0
Embarked      0
Fare          0
Name          0
Parch         0
Pclass        0
Sex           0
SibSp         0
Survived    418
dtype: int64

In [96]:
for dataset in [train, test]:

    
    dataset['Age']      = lambda x: x['Age'].fillna(x['Age'].median(), inplace = True)
    dataset['Embarked'] = lambda x: x['Embarked'].fillna(x['Embarked'].mode(), inplace = True)
    dataset['Fare']     = lambda x: x['Fare'].fillna(x['Fare'].median(), inplace = True)
train.drop(['Cabin', 'PassengerId', 'Ticket'], axis=1, inplace=True)

print("Training set:\n",train.isnull().sum())
print("-" * 10)
print("Test set:\n",test.isnull().sum())

train set
 Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------
test set
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [84]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [83]:
train.shape

(891, 12)

In [81]:
train.size

10692

In [65]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [85]:
train.isnull().sum()

# there is a huge % of Cabin missing from the dataset. I think the best thing to do with this is drop it entirely

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [86]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [87]:
train.Age.head()

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: Age, dtype: float64

In [73]:
train.Cabin.head()

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [74]:
train.Embarked.head()

0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [57]:
# mark train.Age's zero values as NaN
train[['Age']] = train[['Age']].replace(0, np.NaN)

In [58]:
# fill NaN values with mean column values
train.fillna(train.mean(), inplace=True)

In [59]:
# recount NaN's
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [8]:
len(train.columns)

12

In [9]:
len(test.columns)

11

In [11]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [13]:
train[['Fare', 'Survived']].corr()

,Fare,Survived
Fare,1.000000,0.257307
Survived,0.257307,1.000000


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [19]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [25]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [42]:
full_data = pd.concat([train, test], sort=True)
full_data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [54]:
# looks like PassengerId is a drop
full_data.PassengerId.value_counts().head()

1309    1
449     1
431     1
432     1
433     1
Name: PassengerId, dtype: int64

In [55]:
# drop candidate
full_data.Ticket.value_counts()

CA. 2343             11
CA 2144               8
1601                  8
S.O.C. 14879          7
347077                7
3101295               7
PC 17608              7
347082                7
19950                 6
382652                6
347088                6
113781                6
220845                5
PC 17757              5
4133                  5
W./C. 6608            5
349909                5
16966                 5
113503                5
PC 17483              4
2666                  4
W./C. 6607            4
C.A. 33112            4
C.A. 34651            4
17421                 4
113760                4
LINE                  4
C.A. 2315             4
12749                 4
36928                 4
                     ..
110469                1
349911                1
PC 17610              1
2667                  1
349208                1
3101296               1
A.5. 3236             1
367655                1
349241                1
244278                1
2677            

In [58]:
# cabin has too much missing data, will drop
full_data.Cabin.value_counts().head()

C23 C25 C27        6
G6                 5
B57 B59 B63 B66    5
F33                4
C78                4
Name: Cabin, dtype: int64

In [52]:
# looks like most people embarked from Southampton, C = Cherbourg, Q = Queenstown
full_data.Embarked.value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64